# About:

Prepare the data to populate the 'TexteFiscExactCode' column from the 'Textes' table.

In [132]:
# !pip install llama-index
# !pip install llama-index-readers-database
# !pip install llama-index-embeddings-huggingface
# !pip install llama-index-llms-ollama
# !pip install llama-index-postprocessor-cohere-rerank
# !pip install llama-index-postprocessor-flag-embedding-reranker
# !pip install FlagEmbedding
# !pip install openpyxl
# !pip install psycopg2
# !pip install pandas
# !pip install sqlalchemy

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
COUNTRY_NAME = 'MLI Mali'
SPOKEN_LANGUAGE = "French"

In [3]:
from general_config import COUNTRY_NAMES_LIST

# validate COUNTRY_NAME
if COUNTRY_NAME in COUNTRY_NAMES_LIST:
    print('country name OK')

country name OK


## Get data from Postgres

In [26]:
from postgres_connection import get_postgress_data
from sql_files import sql_files
import pandas as pd

### This is not necessary if we extract the data using the llama_index db reader

In [27]:
df = get_postgress_data(sql_files['get_docs_per_country'].replace("%country_name%", COUNTRY_NAME))

/home/andrei/paperless/postgres_connection.py:49: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [28]:
df.head()

,title,content,country
0,img_JO 1997 n°013 (15.07.1997) (SGG),JOURNAL OFFICIEL\n\nDE LA\n\nREPUBLIQUE DU MAL...,MLI Mali
1,img_JO 1972 n°371 (01.03.1972) (SGG),JOURNAL OFFICIEL\nDE LA RÉPUBLIQUE DU MALI\nPA...,MLI Mali
2,img_JO 1997 n°014 (31.07.1997) (SGG),JOURNAL OFFICIEL\n\nDE LA\n\nREPUBLIQUE DU MAL...,MLI Mali
3,img_JO 2011 n°035 (02.09.2011) (SGG),# JOURNAL OFFICIEL\n## DE LA\n## REPUBLIQUE DU...,MLI Mali
4,img_JO 2003 n°001 (10.01.2003) (SGG),JOURNAL OFFICIEL\n\nDE LA\n\nREPUBLIQUE DU MAL...,MLI Mali


### This is mandatory

In [5]:
df_fisc_exacts = get_postgress_data(sql_files['get_textes_fiscaux_exacts'], db='Ferdi')

/home/andrei/paperless/postgres_connection.py:49: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [6]:
df_fisc_exacts.head()

,TexteFiscExactCode,TexteFiscStandardCode,TexteFiscExactCodeCourt,TexteFiscExactComplet
0,_Gen_CGI_CGI,_Gen_CGI,_CGI,Code général des impôts
1,_Gen_CGI_CGImodif,_Gen_CGI,_CGImodif,Modification du Code général des impôts
2,_Gen_CGI_CGImodifModif,_Gen_CGI,_CGImodifModif,Modification de la Modification du Code généra...
3,_Gen_CGI_CGIamendé,_Gen_CGI,_CGIamendé,Code général des impôts amendé
4,_Gen_CGI_CGIappli,_Gen_CGI,_CGIappli,Application du Code général des impôts


# RAG approach

## Get data from Postgres using llama-index db reader

In [24]:
from postgres_connection import psql_conn_config
from llama_index.readers.database import DatabaseReader
from sql_files import sql_files

In [25]:
db = DatabaseReader(
    scheme="postgresql",  # Database Scheme
    host=psql_conn_config.get("HOSTNAME"),  # Database Host
    port="5432",  # Database Port
    user=psql_conn_config.get("USERNAME"),  # Database User
    password=psql_conn_config.get("PASSWORD"),  # Database Password
    dbname=psql_conn_config.get("DATABASE"),  # Database Name
)

### Load the data as llama_index documents

In [139]:
from llama_index.core import Document

documents = db.load_data(query=sql_files['get_docs_MALI'])

In [140]:
documents[0].dict()

{'id_': '2953a3f8-5641-42c7-84d4-856c2756978b',
 'embedding': None,
 'metadata': {},
 'excluded_embed_metadata_keys': [],
 'excluded_llm_metadata_keys': [],
 'relationships': {},
 'text': 'title: img_JO 1997 n°013 (15.07.1997) (SGG), content: JOURNAL OFFICIEL\n\nDE LA\n\nREPUBLIQUE DU MALI\n\n<table>\n  <tr>\n    <th colspan="2">TARIFS DES ABONNEMENTS</th>\n    <th>TARIFS DES INSERTIONS</th>\n    <th>OBSERVATIONS</th>\n  </tr>\n  <tr>\n    <td>1 an</td>\n    <td>6 mois</td>\n    <td rowspan="4">La ligne..................................400 F<br><br>Chaque annonce répétée.......moitié prix<br><br>Il n\'est jamais compté moins de 1.000 F pour les annonces.</td>\n    <td rowspan="4">Prix au numéro de l\'année courante................400F<br>Prix au numéro de l\'année précédente.......450F<br><br>Les demandes d\'abonnement et les annonces doivent être adressées au Secrétariat Général du Gouverne-ment-D.J.O.D.I.J.<br><br>Les abonnements prendront effet à compter de la date de paiement de le

## RAG

### Embeddings model

#### Hugging Face embedding

In [9]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding( model_name="dunzhang/stella_en_1.5B_v5", trust_remote_code=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


#### Ollama embedding

In [10]:
# !pip install llama-index-embeddings-ollama

In [8]:
# from llama_index.embeddings.ollama import OllamaEmbedding
# embed_model = OllamaEmbedding(
#     model_name="llama3.1",
#     base_url="http://localhot:11434",
#     # ollama_additional_kwargs=,
# )

### Vector DataBase

In [10]:
from llama_index.core.node_parser import TokenTextSplitter

chunk_size = 128

transformations_example = [
    TokenTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=int(chunk_size/10),
        separator=" ",
    ),
    embed_model,
]

In [11]:
from llama_index.llms.ollama import Ollama
from llama_index.core import Settings

# setting up the llm
llm = Ollama(model="llama3.1", request_timeout=120.0) 

import os
from llama_index.core import VectorStoreIndex, load_index_from_storage
from llama_index.core.storage import StorageContext
from llama_index.core import Settings

Settings.embed_model = embed_model # we specify the embedding model to be used

if not os.path.exists("storage_MLI Mali"):
    index = VectorStoreIndex.from_documents(documents, transformations=transformations_example)
    # save index to disk
    index.set_index_id("vector_index")
    index.storage_context.persist("./storage_MLI Mali")
else:
    print('loading from local')
    # rebuild storage context
    storage_context = StorageContext.from_defaults(persist_dir="storage_MLI Mali")
    # load index
    index = load_index_from_storage(storage_context, index_id="vector_index")

loading from local


#### Query pipeline

In [12]:
from llama_index.core import PromptTemplate
from llama_index.core.query_pipeline import QueryPipeline

from llama_index.core.response_synthesizers import ResponseMode
from llama_index.core import get_response_synthesizer

from llama_index.core.response_synthesizers import TreeSummarize

In [13]:
retriever = index.as_retriever(similarity_top_k=5)
summarizer = TreeSummarize(llm=llm)

In [14]:
from llama_index.core import get_response_synthesizer
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.query_pipeline import InputComponent

llm = Ollama(model="llama3.1", request_timeout=120.0) 

response_synthesizer = get_response_synthesizer(llm=llm)
retriever_query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)
lang_response = retriever_query_engine.query("Is Mali a French, English, Portuguese or Dutch speaking country? Keep the language only.")

In [15]:
print(str(lang_response))

French.


In [16]:
p = QueryPipeline() #verbose=True)
p.add_modules(
    {
        "input": InputComponent(),
        "retriever": retriever,
        "summarizer": summarizer,
    }
)
p.add_link("input", "retriever")
p.add_link("input", "summarizer", dest_key="query_str")
p.add_link("retriever", "summarizer", dest_key="nodes")

In [18]:
definitions = df_fisc_exacts.set_index(['TexteFiscExactCode'])['TexteFiscExactComplet'].to_dict()

# str(definitions)
# list(definitions.keys())

# ', '.join(list(definitions.keys()))

In [39]:
doc_name = "Tarif douanier CEDEAO 2017 amendé 2020 (DGD)"

def get_text_fisc(doc_name, definitions=definitions, p=p):
    if "JO" in doc_name:
        return None
    else:
        return p.run(input=f"""
                    Consider the content of the document that has this title '{doc_name}', and these definitions: {str(definitions)} 
                    In which of these categories: {', '.join(list(definitions.keys()))}, is it part of? Keep the category only.
                    """)


In [ ]:
output = get_text_fisc(doc_name)
print(str(output))

In [40]:
df_fisc_exacts = df.title.apply(lambda x: get_text_fisc(x))

In [41]:
df['TexteFiscExactCode'] = df_fisc_exacts

In [42]:
df[['title', 'country', 'TexteFiscExactCode']].to_excel("output/TexteFiscExactCode2.xlsx")

In [38]:
definitions

{'_Gen_CGI_CGI': 'Code général des impôts',
 '_Gen_CGI_CGImodif': 'Modification du Code général des impôts',
 '_Gen_CGI_CGImodifModif': 'Modification de la Modification du Code général des impôts',
 '_Gen_CGI_CGIamendé': 'Code général des impôts amendé',
 '_Gen_CGI_CGIappli': 'Application du Code général des impôts',
 '_Gen_CGI_CGIappliModif': "Modification de l'Application du Code général des impôts",
 '_Gen_CGI_CGIsuppl': 'Supplément au Code général des impôts',
 '_Gen_CGI_CGIsupplAppli': 'Application du Supplément au Code général des impôts',
 '_Gen_CGI_CGIone': 'Code des impôts directs et indirects',
 '_Gen_CGI_CGIoneModif': 'Modification du Code des impôts directs et indirects',
 '_Gen_CGI_CGIoneAmendé': 'Code des impôts directs et indirects amendé',
 '_Gen_CGI_CGIbis': "Code de l'enregistrement, du timbre et de l'impôt sur le revenu des capitaux mobiliers",
 '_Gen_CGI_CGIbisModif': "Modification du Code de l'enregistrement, du timbre et de l'impôt sur le revenu des capitaux mobil

#### Other Query pipeline trials

In [ ]:
# try chaining basic prompts
# prompt_str = "What's the date of the document that has this title '{doc_title}' ? please respond in english."
prompt_str = "Based on the title and the content of this text '{doc_title}', in which of these categories: legislation, reglementation or convention, is it part of? please reply in French."
prompt_tmpl = PromptTemplate(prompt_str)

p = QueryPipeline(chain=[prompt_tmpl, llm], verbose=True)

In [ ]:
# generate question regarding topic
prompt_str1 = "Consider the title and the content of this text '{doc_title}', which is a legal document from Mali."
prompt_tmpl1 = PromptTemplate(prompt_str1)
# use HyDE to hallucinate answer.
prompt_str2 = (
    # "Please consider these definitions:\n"
    # "If it's a decree, then it's legislation."
    # "If it's an ordinance then it's a regulation."
    # "If it's a law then it's a legislation."
    # "If it's an international law it usually is a convention"
    "\n"
    "{response1}\n"
    "in which of these categories: legislation, regulation or convention, is it part of? keep the category only."
)
prompt_tmpl2 = PromptTemplate(prompt_str2)

# llm = OpenAI(model="gpt-3.5-turbo")
retriever = index.as_retriever(retriever_mode=llm, similarity_top_k=5)
p = QueryPipeline(
    chain=[prompt_tmpl1, llm, prompt_tmpl2, llm, retriever], verbose=True
)


In [144]:
output = p.run(input=f"""
               Consider the content of the document that has this title '{doc_name}', and these definitions: {french_definitions} 
               In which of these categories: {', '.join(textes_legaux_exacts)}, is it part of?
               """)

print(str(output))

> Running module 65c2142d-3655-4e00-90d5-438551962fbc with input: 
doc_title: 
               Consider the content of the document that has this title 'Tarif douanier CEDEAO 2017 amendé 2020 (DGD)', and these definitions: 
Loi: A law passed by the legislature (parliament) in a ...

> Running module dff56ffc-71f2-4f80-ad2b-6c941cfaa8d7 with input: 
messages: Consider the title and the content of this text '
               Consider the content of the document that has this title 'Tarif douanier CEDEAO 2017 amendé 2020 (DGD)', and these definitions: 
Loi: A...

> Running module 206d3771-9dae-457a-a72e-1052551c99b9 with input: 
response1: assistant: Based on the content and definitions provided, I would categorize "Tarif douanier CEDEAO 2017 amendé 2020 (DGD)" as follows:

The title suggests that this document is related to tariffs or ...

> Running module 3722a567-c7d1-4b5e-be7b-8677ee044d17 with input: 
messages: 
Based on the content and definitions provided, I would categorize "Tarif d

In [ ]:
df_texteLegaleExactCode.to_excel("texteLegaleExactCode.xlsx")